In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_A6_011, input_features_A6_011 = pre_models.Input_A6_011('weights/')
    
    outputs = ['Input_A6_011']

    preprocessings = ['min_max_normalization']

    models = [model_A6_011]

    input_features = [input_features_A6_011]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=7
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A6_011.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_135', 'Input_C_059', 'Input_A5_011', 'Input_C_056',
       'Input_C_053', 'Input_A3_011', 'Input_A2_011'], dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [8]:
nn_ave(nn, y)

test_rmse_ave: 0.004319068447480295
[0.006094242301827042, 0.001665057363213684, 0.0013048397852114865, 0.011905717529131285, 0.0013036474479977123, 0.001434631512299441, 0.0014329224592242156, 0.012416037844995814, 0.0009611172825076823, 0.0046724709483945805]


test_r2_ave: -5.016577302074985
[-6.2550218164388856, 0.4884456050730035, 0.7433308626985382, -20.276679893057636, 0.7314478481573852, 0.653149638093113, 0.4975776682481423, -24.415503491302612, 0.8228847428939872, -3.155404185114887]


## Linear Regression Baseline

In [14]:
lr_rmse_ave_fea(skew_data,7)

train_rmse: 0.0011348670130296502
test_rmse: 0.0011442370831728332
test_r2: 0.762888913734182


## Lasso

In [15]:
laso_rmse_ave(skew_data[title],y,5e-06)

test_rmse_ave: 0.0011455083316005306
[0.0009891828415996768, 0.0011267510198912306, 0.0012831399439415872, 0.0010615718659504148, 0.0011672294627778493, 0.001278031888481265, 0.0011002660021856895, 0.0013052211561260792, 0.0010858966368036843, 0.001057792498247831]


test_r2_ave: 0.7650543173640052
[0.8088596236180619, 0.7657447462702572, 0.7517968321708633, 0.8308424013530976, 0.7847115149793393, 0.7247389259500123, 0.7037768863784355, 0.7191328991165982, 0.7739106128296445, 0.7870287309737416]


## ElasticNet

In [16]:
ElasticNet_rmse_ave(skew_data[title],y,[0.008],0.01)

test_rmse_ave: 0.0011619151531082903
[0.0010200766420792506, 0.001127882071265642, 0.0013253486056828926, 0.0011113942618323368, 0.0011871222124048221, 0.0012780009522357948, 0.0010978636817376476, 0.0013228313688907894, 0.0010926846618543723, 0.001055947073099355]


test_r2_ave: 0.758928016399344
[0.7967339282807036, 0.7652742116002362, 0.7351990604150449, 0.8145917686053841, 0.7773107865069462, 0.7247522518163972, 0.7050690211223541, 0.7115027802040559, 0.7710751731791423, 0.7877711822631773]


## SVR

In [17]:
model=SVR(kernel='rbf', C=1.25, gamma= 5e-06, epsilon= 0)
svr_rmse_ave(skew_data,7,model)

train_rmse: 0.0011552584765401213
test_rmse: 0.0011443004043795043
test_r2: 0.7636970298066595


## XGB

In [18]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 4, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.65,
              colsample_bytree = 0.4,
              reg_alpha =  0,
              reg_lambda = 1)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.0011255591644585131
[0.0009460494864920952, 0.0010964265120617356, 0.0012739661243948664, 0.0010336909052578648, 0.0011823387412066568, 0.0012736759726907337, 0.0010680446743271046, 0.0013530920585090418, 0.001002408079341646, 0.0010258990903033853]


test_r2_ave: 0.7730050673049105
[0.8251655560767585, 0.7781842004773261, 0.7553332063155976, 0.8396111766541221, 0.7791018089943276, 0.726612072888036, 0.7208726497300073, 0.6981526603370963, 0.8073396622361051, 0.7996776793397282]


## RandomForest

In [19]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 3,
            min_samples_split = 3,
            max_leaf_nodes = 3,
            min_samples_leaf =7,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.001118916309689132
[0.0009605490362739144, 0.0011150928649105627, 0.001338740331599537, 0.0010678791429009514, 0.0011567087653269292, 0.0012317033203886045, 0.0010273334403260274, 0.0012864929526926194, 0.00099859973453773, 0.0010060635079344434]


test_r2_ave: 0.7766919809511834
[0.8197653161248184, 0.7705672060123969, 0.7298207663761055, 0.8288263468683351, 0.7885749839678051, 0.7443336070870843, 0.7417463876005321, 0.7271352183672792, 0.8088007901818656, 0.8073491869256116]


In [9]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=5e-06)
elastic_mod=ElasticNet(alpha=[0.008], l1_ratio=0.01)
svr_mod=SVR(kernel='rbf', C=1.25, gamma= 5e-06, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 4, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.65,
              colsample_bytree = 0.4,
              reg_alpha =  0,
              reg_lambda = 1)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 3,
            min_samples_split = 3,
            max_leaf_nodes = 3,
            min_samples_leaf =7,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.0011196107795710072
[0.0009585864595784774, 0.0010903903942567315, 0.0012923656153281254, 0.0010573076054636193, 0.00115350043889805, 0.001250044627694648, 0.0010493655795137092, 0.0012890907365861624, 0.0010390287685438212, 0.0010164275698467272]


test_r2_ave: 0.7760891241448983
[0.8205010683149705, 0.7806197869314648, 0.7482148803210689, 0.83219866053183, 0.7897462031527566, 0.7366626530809454, 0.7305506224522398, 0.7260321273494819, 0.7930057195522665, 0.8033595197619575]


In [10]:

stack_mod = StackingRegressor(regressors=[svr_mod,random_mod,lasso_mod, vote_mod], 
                           meta_regressor=xgb_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.001183250381056081
[0.0009948293478069332, 0.0011536926539622645, 0.0013251097816936773, 0.0010977457257476887, 0.0011895795993575856, 0.0013800037291015607, 0.0011322454749689195, 0.0014033723356070935, 0.0010872395846201267, 0.001068685577694962]


test_r2_ave: 0.7488521733309715
[0.8066712401066798, 0.7544083060305837, 0.7352944845450622, 0.8191176380300556, 0.776387882430327, 0.6790614103577641, 0.6863070613543465, 0.6753028441000701, 0.7733510494448179, 0.782619816910008]


In [11]:
ave(skew_data[title],y,0.1,stack_mod,0.6,vote_mod,0.3,random_mod)

test_rmse_ave: 0.0011132137596788598
[0.000947339090470658, 0.0010861907835099471, 0.001299732520310997, 0.0010503637838153866, 0.0011461361296933265, 0.0012481361994454107, 0.0010398785253714457, 0.001287742738442437, 0.0010222919473399385, 0.001004325878389051]


test_r2_ave: 0.7786255181467501
[0.8246885813093808, 0.7823064072491501, 0.7453361844900787, 0.8343954789878758, 0.7924222859117305, 0.7374661072845794, 0.7354006490441651, 0.7266048024623999, 0.7996205961253401, 0.8080140886028006]


In [13]:
name='Input_A6_011'
test_pred=[]
seed=[0,1,3,8,9,6,2,5]
stack_w=0.1
vote_w=0.6
model_w=0.3
model=random_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)


final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))
